In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
dsf1 = tabula.read_pdf('Topcondoimage 05-09-2022.pdf', pages='all',lattice=True)
dsf2 = tabula.read_pdf('Topcondoimage 05-16-2022.pdf', pages='all',lattice=True)
dsf3 = tabula.read_pdf('Topcondoimage 05-23-2022.pdf', pages='all',lattice=True)
dsf4 = tabula.read_pdf('Topcondoimage 05-30-2022.pdf', pages='all',lattice=True)
dsf5 = tabula.read_pdf('Topcondoimage 06-06-2022.pdf', pages='all',lattice=True)

In [3]:
df1 = dsf1[0]
df2 = dsf2[0]
df3 = dsf3[0]
df4 = dsf4[0]
df5 = dsf5[0]

In [4]:
df1.columns = df1.iloc[0]
df2.columns = df2.iloc[0]
df3.columns = df3.iloc[0]
df4.columns = df4.iloc[0]
df5.columns = df5.iloc[0]

In [5]:
df1 = df1.drop(df1.index[0])
df2 = df2.drop(df2.index[0])
df3 = df3.drop(df3.index[0])
df4 = df4.drop(df4.index[0])
df5 = df5.drop(df5.index[0])

In [6]:
df1 = df1.iloc[: , 1:]
df2 = df2.iloc[: , 1:]
df3 = df3.iloc[: , 1:]
df4 = df4.iloc[: , 1:]
df5 = df5.iloc[: , 1:]

In [7]:
df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [8]:
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [9]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [10]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only'],
      dtype='object')

In [11]:
x = df['Close Date'].str.startswith('5')
df = df[x]

In [12]:
len(df)

45

In [13]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only
0,Faena House\r3315 Collins Ave 6a\rMiami Beach,1/31/2022,5/4/2022,63,"$15,500,000.00",6246,"$3,276.96",Toni Schrager,Brown Harris Stevens,Bryan Sereny PA,Douglas Elliman,Faena House 3315 Collins Ave 6a Miami Beach,Faena House,3315 Collins Ave 6a Miami Beach
1,Santa Maria\r1643 Brickell Ave PH 4902\rMiami,5/10/2021,5/5/2022,309,"$12,000,000.00",10000,"$1,200.00",Audrey Ross,"Compass Florida, LLC",Liz Hogan,"Compass Florida, LLC",Santa Maria 1643 Brickell Ave PH 4902 Miami,Santa Maria,1643 Brickell Ave PH 4902 Miami
2,Oceana Bal Harbour Condo\r10201 Collins Ave 21...,9/22/2021,5/5/2022,201,"$8,900,000.00",3264,"$2,726.72",Jonathan Mann,Coldwell Banker Realty,Omer Horev,Charles Rutenberg Realty,Oceana Bal Harbour Condo 10201 Collins Ave 210...,Oceana Bal Harbour Condo,10201 Collins Ave 2103 Bal Harbour
3,Oceana Bal Harbour Condo\r10203 Collins Ave 16...,5/1/2022,5/2/2022,1,"$8,900,000.00",3450,"$2,579.71",Comp Sale,Comp Sale,Lourdes Alatriste,Douglas Elliman,Oceana Bal Harbour Condo 10203 Collins Ave 160...,Oceana Bal Harbour Condo,10203 Collins Ave 1602 Bal Harbour
4,Continuum on South Beach\r50 S Pointe Dr 3104\...,4/6/2022,5/2/2022,2,"$8,800,000.00",2173,"$4,049.70",Jorge Martinez,One Sotheby's Intl Realty,Devin Kay,Douglas Elliman,Continuum on South Beach 50 S Pointe Dr 3104 M...,Continuum on South Beach,50 S Pointe Dr 3104 Miami Beach
5,Oceana Key Biscayne\r360 Ocean Dr 501S\rKey Bi...,8/11/2021,5/5/2022,253,"$8,800,000.00",3791,"$2,321.29",Brigitte Nachtigall,Great Properties Intl,David Deleon,"Compass Florida, LLC",Oceana Key Biscayne 360 Ocean Dr 501S Key Bisc...,Oceana Key Biscayne,360 Ocean Dr 501S Key Biscayne
6,One Thousand Museum\r1000 E Biscayne Blvd 3201...,4/26/2022,5/4/2022,1,"$7,300,000.00",4600,"$1,586.96",Patrice Hallot,London Foster Realty,Anna Sherrill,One Sotheby's Intl Realty,One Thousand Museum 1000 E Biscayne Blvd 3201 ...,One Thousand Museum,1000 E Biscayne Blvd 3201 Miami
7,Cloisters on the Bay Condo\r3471 Main Hwy 727\...,2/28/2022,5/2/2022,7,"$7,000,000.00",5739,"$1,219.72",Ilana Levitt,"Compass Florida, LLC",Ilana Levitt,"Compass Florida, LLC",Cloisters on the Bay Condo 3471 Main Hwy 727 M...,Cloisters on the Bay Condo,3471 Main Hwy 727 Miami
8,Bayview at Fisher Island\r5132 Fisher Island D...,11/16/2021,5/2/2022,121,"$6,500,000.00",3580,"$1,815.64",Riley Smith,BHHS EWM Realty,Riley Smith,BHHS EWM Realty,Bayview at Fisher Island 5132 Fisher Island Dr...,Bayview at Fisher Island,5132 Fisher Island Dr 5132 Miami Beach
9,P0orsche Design Tower\r18555 Collins Ave 2501\...,1/11/2022,5/2/2022,51,"$6,050,000.00",5913,"$1,340.57",Enrique Tettamanti,Dezer Platinum Realty,Melissa Barragan,Dezer Platinum Realty,P0orsche Design Tower 18555 Collins Ave 2501 S...,P,0orsche Design Tower 18555 Collins Ave 2501 Su...


In [ ]:
df.loc[len(df.index)] = ['The Surf Club 9001 Collins Avenue S-507','']

In [16]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc'],
      dtype='object')

In [14]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [15]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

<ipython-input-15-4267de5cdfa3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)


In [161]:
df['loc']

0     (FAENA HOUSE, 3315 Collins Ave #6a, Miami Beac...
1     (1643 Brickell Ave # 4902, Miami, FL 33129, US...
2     (10201 Collins Ave Unit 2103, Bal Harbour, FL ...
3     (10203 Collins Ave Unit 1602, Bal Harbour, FL ...
4     (50 South Pointe Dr APT 3104, Miami Beach, FL ...
5     (360 Ocean Dr #501s, Key Biscayne, FL 33149, U...
6     (1000 Biscayne Blvd #3201, Miami, FL 33132, US...
7     (3471 Main Hwy APT 727, Miami, FL 33133, USA, ...
8     (5132 Fisher Island Dr #5132, Miami Beach, FL ...
9     (18555 Collins Ave, Sunny Isles Beach, FL 3316...
10    (88 SW 7th St #4201, Miami, FL 33130, USA, (25...
11    (5500 Island Estates Dr, Aventura, FL 33160, U...
12    (1425 Brickell Ave #58e, Miami, FL 33131, USA,...
13    (2423 Fisher Island Dr #5203, Miami Beach, FL ...
14    (400 Alton Rd APT 2506, Miami Beach, FL 33139,...
15    (1100 Biscayne Blvd UNIT 3507, Miami, FL 33132...
16    (5500 Island Estates Dr #1106, Aventura, FL 33...
17    (1000 South Pointe Dr APT 101, Miami Beach

In [162]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

<ipython-input-162-823ae924bbc1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)


Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [163]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point'],
      dtype='object')

In [164]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)



C:\Users\aefar\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [165]:
df.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 42, 43, 44, 47, 49],
           dtype='int64')

In [192]:
### Not sure why this works the way it does. But if I want to simply reset the index because the numbers are askew,
### I need to set both 'drop' and 'inplace' to 'True'
df.reset_index(drop=True, inplace=True)

In [193]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,...,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int,year
0,Faena House 3315 Collins Ave 6a Miami Beach,1/31/2022,5/4/2022,63,"$15,500,000.00",6246,"$3,276.96",Toni Schrager,Brown Harris Stevens,Bryan Sereny PA,...,Faena House,3315 Collins Ave 6a Miami Beach,"(FAENA HOUSE, 3315 Collins Ave #6a, Miami Beac...","(25.8078488, -80.1232302, 0.0)",25.807849,-80.123230,0.0,15500000.0,3276.96,2021
1,Santa Maria 1643 Brickell Ave PH 4902 Miami,5/10/2021,5/5/2022,309,"$12,000,000.00",10000,"$1,200.00",Audrey Ross,"Compass Florida, LLC",Liz Hogan,...,Santa Maria,1643 Brickell Ave PH 4902 Miami,"(1643 Brickell Ave # 4902, Miami, FL 33129, US...","(25.755254, -80.194744, 0.0)",25.755254,-80.194744,0.0,12000000.0,1200.00,2021
2,Oceana Bal Harbour Condo 10201 Collins Ave 210...,9/22/2021,5/5/2022,201,"$8,900,000.00",3264,"$2,726.72",Jonathan Mann,Coldwell Banker Realty,Omer Horev,...,Oceana Bal Harbour Condo,10201 Collins Ave 2103 Bal Harbour,"(10201 Collins Ave Unit 2103, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0,8900000.0,2726.72,2021
3,Oceana Bal Harbour Condo 10203 Collins Ave 160...,5/1/2022,5/2/2022,1,"$8,900,000.00",3450,"$2,579.71",Comp Sale,Comp Sale,Lourdes Alatriste,...,Oceana Bal Harbour Condo,10203 Collins Ave 1602 Bal Harbour,"(10203 Collins Ave Unit 1602, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0,8900000.0,2579.71,2021
4,Continuum on South Beach 50 S Pointe Dr 3104 M...,4/6/2022,5/2/2022,2,"$8,800,000.00",2173,"$4,049.70",Jorge Martinez,One Sotheby's Intl Realty,Devin Kay,...,Continuum on South Beach,50 S Pointe Dr 3104 Miami Beach,"(50 South Pointe Dr APT 3104, Miami Beach, FL ...","(25.7682956, -80.1316247, 0.0)",25.768296,-80.131625,0.0,8800000.0,4049.70,2021
5,Oceana Key Biscayne 360 Ocean Dr 501S Key Bisc...,8/11/2021,5/5/2022,253,"$8,800,000.00",3791,"$2,321.29",Brigitte Nachtigall,Great Properties Intl,David Deleon,...,Oceana Key Biscayne,360 Ocean Dr 501S Key Biscayne,"(360 Ocean Dr #501s, Key Biscayne, FL 33149, U...","(25.6942877, -80.1578218, 0.0)",25.694288,-80.157822,0.0,8800000.0,2321.29,2021
6,One Thousand Museum 1000 E Biscayne Blvd 3201 ...,4/26/2022,5/4/2022,1,"$7,300,000.00",4600,"$1,586.96",Patrice Hallot,London Foster Realty,Anna Sherrill,...,One Thousand Museum,1000 E Biscayne Blvd 3201 Miami,"(1000 Biscayne Blvd #3201, Miami, FL 33132, US...","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0,7300000.0,1586.96,2021
7,Cloisters on the Bay Condo 3471 Main Hwy 727 M...,2/28/2022,5/2/2022,7,"$7,000,000.00",5739,"$1,219.72",Ilana Levitt,"Compass Florida, LLC",Ilana Levitt,...,Cloisters on the Bay Condo,3471 Main Hwy 727 Miami,"(3471 Main Hwy APT 727, Miami, FL 33133, USA, ...","(25.7260243, -80.2424968, 0.0)",25.726024,-80.242497,0.0,7000000.0,1219.72,2021
8,Bayview at Fisher Island 5132 Fisher Island Dr...,11/16/2021,5/2/2022,121,"$6,500,000.00",3580,"$1,815.64",Riley Smith,BHHS EWM Realty,Riley Smith,...,Bayview at Fisher Island,5132 Fisher Island Dr 5132 Miami Beach,"(5132 Fisher Island Dr #5132, Miami Beach, FL ...","(25.7620721, -80.1477688, 0.0)",25.762072,-80.147769,0.0,6500000.0,1815.64,2021
9,P0orsche Design Tower 18555 Collins Ave 2501 S...,1/11/2022,5/2/2022,51,"$6,050,000.00",5913,"$1,340.57",Enrique Tettamanti,Dezer Platinum Realty,Melissa Barragan,...,P,0orsche Design Tower 18555 Collins Ave 2501 Su...,"(18555 Collins Ave, Sunny Isles Beach, FL 3316...","(25.9482145, -80.1197839, 0.0)",25.948214,-80.119784,0.0,6050000.0,1340.57,2021


In [197]:
df.at[9,'building_name']=('Porsche Design Tower')
df.at[19,'building_name']=('Grove Isle')
df.at[29,'building_name']=('W South Beach')
df.at[39,'building_name']=('Jade Ocean')
df.at[44,'building_name']=('Oceana')

df.at[20,'Agent']=('Bill Hernandez')

df.at[35,'Buyer Agent']=('Mendel Fellig')

df.at[29,'address_only']=('2201 Collins Ave 804')

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [198]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'May 2022 Top Miami-Dade Condo Sales ')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['Sq. Ft.'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Address: {bold_start}{info[-1]}{bold_end}{new_line}\
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Square Feet: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

m

In [205]:
m.save('index.html')

In [171]:
### Clean DF for graphic ###

In [172]:
df['Sale Price int'] = df['Sale Price']
df['price_per_sqft_int'] = df['price_per_sqft']

<ipython-input-172-347b4b31049c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price']
<ipython-input-172-347b4b31049c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft_int'] = df['price_per_sqft']


In [173]:
df['Sale Price int'] = df['Sale Price int'].replace('\$','',regex=True)
df['Sale Price int'] = df['Sale Price int'].replace(',','',regex=True)

df['price_per_sqft_int'] = df['price_per_sqft_int'].replace('\$','',regex=True)
df['price_per_sqft_int'] = df['price_per_sqft_int'].replace(',','',regex=True)

<ipython-input-173-6b4fff0128f6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price int'].replace('\$','',regex=True)
<ipython-input-173-6b4fff0128f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = df['Sale Price int'].replace(',','',regex=True)
<ipython-input-173-6b4fff0128f6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [174]:
df['Sale Price int'] = pd.to_numeric(df['Sale Price int'])
df['price_per_sqft_int'] = pd.to_numeric(df['price_per_sqft_int'])

<ipython-input-174-2f247dbc6c1d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sale Price int'] = pd.to_numeric(df['Sale Price int'])
<ipython-input-174-2f247dbc6c1d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_per_sqft_int'] = pd.to_numeric(df['price_per_sqft_int'])


In [175]:
df['days_on_market'] = pd.to_numeric(df['days_on_market'])
df['Sq. Ft.'] = pd.to_numeric(df['Sq. Ft.'])

<ipython-input-175-f436ebf6f88e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['days_on_market'] = pd.to_numeric(df['days_on_market'])
<ipython-input-175-f436ebf6f88e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sq. Ft.'] = pd.to_numeric(df['Sq. Ft.'])


In [176]:
df['year'] = 2022

<ipython-input-176-581b795327a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = 2022


In [177]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude',
       'Sale Price int', 'price_per_sqft_int', 'year'],
      dtype='object')

In [204]:
df.sort_values('Sale Price int',ascending=False)

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,...,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int,year
0,Faena House 3315 Collins Ave 6a Miami Beach,1/31/2022,5/4/2022,63,"$15,500,000.00",6246,"$3,276.96",Toni Schrager,Brown Harris Stevens,Bryan Sereny PA,...,Faena House,3315 Collins Ave 6a Miami Beach,"(FAENA HOUSE, 3315 Collins Ave #6a, Miami Beac...","(25.8078488, -80.1232302, 0.0)",25.807849,-80.123230,0.0,15500000.0,3276.96,2021
20,Apogee Condo 800 S Pointe Dr 1401 Miami Beach,1/26/2021,5/16/2022,360,"$12,250,000.00",6595,"$2,948.96",Bill Hernandez,Douglas Elliman,Clemente Sanchez PA,...,Apogee Condo,800 S Pointe Dr 1401 Miami Beach,"(800 South Pointe Dr APT 1401, Miami Beach, FL...","(25.7677245, -80.13634490000001, 0.0)",25.767724,-80.136345,0.0,12250000.0,2948.96,2021
21,Mansions at Acqualina 17749 Collins Ave 2401/2...,12/25/2021,5/18/2022,99,"$12,210,000.00",9218,"$1,324.58",Boris Vertsberger,"FIP Realty Services, LLC",Chad Carroll,...,Mansions at Acqualina,17749 Collins Ave 2401/2402 Sunny Isles Beach,"(17749 Collins Ave, Sunny Isles Beach, FL 3316...","(25.9400648, -80.12026379999999, 0.0)",25.940065,-80.120264,0.0,12210000.0,1324.58,2021
1,Santa Maria 1643 Brickell Ave PH 4902 Miami,5/10/2021,5/5/2022,309,"$12,000,000.00",10000,"$1,200.00",Audrey Ross,"Compass Florida, LLC",Liz Hogan,...,Santa Maria,1643 Brickell Ave PH 4902 Miami,"(1643 Brickell Ave # 4902, Miami, FL 33129, US...","(25.755254, -80.194744, 0.0)",25.755254,-80.194744,0.0,12000000.0,1200.00,2021
22,Fisher Island 7046 Fisher Island Dr 7046 Miami...,5/28/2021,5/17/2022,266,"$10,400,000.00",3835,"$2,711.86",Karla Abaunza,Luxury Living Realty,Jill Eber,...,Fisher Island,7046 Fisher Island Dr 7046 Miami Beach,"(7046 Fisher Island Dr #7046, Miami Beach, FL ...","(25.7623526, -80.1473326, 0.0)",25.762353,-80.147333,0.0,10400000.0,2711.86,2021
23,Regalia 19575 Collins Ave 38 Sunny Isles Beach,2/4/2022,5/17/2022,35,"$10,150,000.00",7600,"$1,840.44",Chad Carroll,"Compass Florida, LLC",Roberto Morales,...,Regalia,19575 Collins Ave 38 Sunny Isles Beach,"(19575 Collins Ave UNIT 38, Sunny Isles Beach,...","(25.956929, -80.1193868, 0.0)",25.956929,-80.119387,0.0,10150000.0,1840.44,2021
30,Fendi Chateau 9349 Collins Ave 402 Surfside,3/24/2022,5/25/2022,14,"$9,500,000.00",3453,"$2,751.23",Romina Grinberg,Brown Harris Stevens,Ximena Penuela,...,Fendi Chateau,9349 Collins Ave 402 Surfside,"(9349 Collins Ave Apt 402, Surfside, FL 33154,...","(25.8830205, -80.1216893, 0.0)",25.883021,-80.121689,0.0,9500000.0,2751.23,2021
3,Oceana Bal Harbour Condo 10203 Collins Ave 160...,5/1/2022,5/2/2022,1,"$8,900,000.00",3450,"$2,579.71",Comp Sale,Comp Sale,Lourdes Alatriste,...,Oceana Bal Harbour Condo,10203 Collins Ave 1602 Bal Harbour,"(10203 Collins Ave Unit 1602, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0,8900000.0,2579.71,2021
2,Oceana Bal Harbour Condo 10201 Collins Ave 210...,9/22/2021,5/5/2022,201,"$8,900,000.00",3264,"$2,726.72",Jonathan Mann,Coldwell Banker Realty,Omer Horev,...,Oceana Bal Harbour Condo,10201 Collins Ave 2103 Bal Harbour,"(10201 Collins Ave Unit 2103, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0,8900000.0,2726.72,2021
4,Continuum on South Beach 50 S Pointe Dr 3104 M...,4/6/2022,5/2/2022,2,"$8,800,000.00",2173,"$4,049.70",Jorge Martinez,One Sotheby's Intl Realty,Devin Kay,...,Continuum on South Beach,50 S Pointe Dr 3104 Miami Beach,"(50 South Pointe Dr APT 3104, Miami Beach, FL ...","(25.7682956, -80.1316247, 0.0)",25.768296,-80.131625,0.0,8800000.0,4049.70,2021


In [196]:
df.sort_values('days_on_market',ascending=False)
#df.sort_values('days_on_market',ascending=True)

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,...,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int,year
16,Prive Condo 5500 Island Estates Dr 1106 Aventura,2/19/2020,5/12/2022,775,"$3,200,000.00",2877,"$1,112.27",Jonathan Lief,Aventura Real Estate,Nicole Holtzheuser,...,Prive Condo,5500 Island Estates Dr 1106 Aventura,"(5500 Island Estates Dr #1106, Aventura, FL 33...","(25.946531, -80.130636, 0.0)",25.946531,-80.130636,0.0,3200000.0,1112.27,2021
11,Prive Condo 5500 Island Estates Dr PH7 Aventura,8/17/2020,5/12/2022,561,"$5,800,000.00",4417,"$1,313.11",Bento Queiroz,"Compass Florida, LLC",Diana Shay,...,Prive Condo,5500 Island Estates Dr PH7 Aventura,"(5500 Island Estates Dr, Aventura, FL 33160, U...","(25.946531, -80.130636, 0.0)",25.946531,-80.130636,0.0,5800000.0,1313.11,2021
26,Residences by Armani Casa 18975 Collins Ave 15...,10/19/2020,5/18/2022,540,"$5,200,000.00",3748,"$1,387.41",Melissa Barragan,Dezer Platinum Realty LLC,Neal Klotsman,...,Residences by Armani Casa,18975 Collins Ave 1500 Sunny Isles Beach,"(18975 Collins Ave #1500, Sunny Isles Beach, F...","(25.9529783, -80.1191546, 0.0)",25.952978,-80.119155,0.0,5200000.0,1387.41,2021
43,Residences by Armani Casa 18975 Collins Ave 46...,1/28/2021,5/31/2022,449,"$4,550,000.00",2682,"$1,696.50",Mariana Niro,Douglas Elliman,Christian Espinosa,...,Residences by Armani Casa,18975 Collins Ave 4602 Sunny Isles Beach,"(18975 Collins Ave #4602, Sunny Isles Beach, F...","(25.9529783, -80.1191546, 0.0)",25.952978,-80.119155,0.0,4550000.0,1696.50,2021
20,Apogee Condo 800 S Pointe Dr 1401 Miami Beach,1/26/2021,5/16/2022,360,"$12,250,000.00",6595,"$2,948.96",Billy Hernandez,Douglas Elliman,Clemente Sanchez PA,...,Apogee Condo,800 S Pointe Dr 1401 Miami Beach,"(800 South Pointe Dr APT 1401, Miami Beach, FL...","(25.7677245, -80.13634490000001, 0.0)",25.767724,-80.136345,0.0,12250000.0,2948.96,2021
27,Eighty Seven Park 8701 Collins Ave 706 Miami B...,6/2/2021,5/18/2022,327,"$4,875,000.00",3206,"$2,749.58",Diana Garchitorena,Douglas Elliman,Diana Garchitorena,...,Eighty Seven Park,8701 Collins Ave 706 Miami Beach,"(8701 Collins Ave #706, Miami Beach, FL 33154,...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0,4875000.0,2749.58,2021
1,Santa Maria 1643 Brickell Ave PH 4902 Miami,5/10/2021,5/5/2022,309,"$12,000,000.00",10000,"$1,200.00",Audrey Ross,"Compass Florida, LLC",Liz Hogan,...,Santa Maria,1643 Brickell Ave PH 4902 Miami,"(1643 Brickell Ave # 4902, Miami, FL 33129, US...","(25.755254, -80.194744, 0.0)",25.755254,-80.194744,0.0,12000000.0,1200.00,2021
22,Fisher Island 7046 Fisher Island Dr 7046 Miami...,5/28/2021,5/17/2022,266,"$10,400,000.00",3835,"$2,711.86",Karla Abaunza,Luxury Living Realty,Jill Eber,...,Fisher Island,7046 Fisher Island Dr 7046 Miami Beach,"(7046 Fisher Island Dr #7046, Miami Beach, FL ...","(25.7623526, -80.1473326, 0.0)",25.762353,-80.147333,0.0,10400000.0,2711.86,2021
5,Oceana Key Biscayne 360 Ocean Dr 501S Key Bisc...,8/11/2021,5/5/2022,253,"$8,800,000.00",3791,"$2,321.29",Brigitte Nachtigall,Great Properties Intl,David Deleon,...,Oceana Key Biscayne,360 Ocean Dr 501S Key Biscayne,"(360 Ocean Dr #501s, Key Biscayne, FL 33149, U...","(25.6942877, -80.1578218, 0.0)",25.694288,-80.157822,0.0,8800000.0,2321.29,2021
33,St Regis Bal Harbour 9705 Collins Ave 403N Bal...,9/2/2021,5/24/2022,204,"$5,100,000.00",2840,"$1,795.77",Ricardo Montalvan,"Weichert Realtors, NexGen",NON-MLS MEMBER,...,St Regis Bal Harbour,9705 Collins Ave 403N Bal Harbour,"(9705 Collins Ave #403n, Bal Harbour, FL 33154...","(25.8892229, -80.1228812, 0.0)",25.889223,-80.122881,0.0,5100000.0,1795.77,2021


### Sale price sum

In [180]:
"${:,.2f}".format(df['Sale Price int'].sum())

'$284,110,500.00'

### Sale price average

In [181]:
"${:,.2f}".format(df['Sale Price int'].mean())

'$6,313,566.67'

### Price per sf average

In [182]:
"${:,.2f}".format(df['price_per_sqft_int'].mean())

'$1,907.26'

### Price per square fot min vs. max

In [183]:
"${:,.2f}".format(df['price_per_sqft_int'].max(axis=0))

'$4,049.70'

In [184]:
"${:,.2f}".format(df['price_per_sqft_int'].min(axis=0))

'$801.28'

### Sale price min and max

In [185]:
"${:,.2f}".format(df['Sale Price int'].max(axis=0))

'$15,500,000.00'

In [186]:
"${:,.2f}".format(df['Sale Price int'].min(axis=0))

'$2,070,000.00'

In [187]:
len(df)

45

In [188]:
df['building_address_city'] = df['building_address_city'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df

<ipython-input-188-10788a26de8e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['building_address_city'] = df['building_address_city'].map(lambda x: re.sub(r'\r', ' ', x))


,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,...,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int,year
0,Faena House 3315 Collins Ave 6a Miami Beach,1/31/2022,5/4/2022,63,"$15,500,000.00",6246,"$3,276.96",Toni Schrager,Brown Harris Stevens,Bryan Sereny PA,...,Faena House,3315 Collins Ave 6a Miami Beach,"(FAENA HOUSE, 3315 Collins Ave #6a, Miami Beac...","(25.8078488, -80.1232302, 0.0)",25.807849,-80.123230,0.0,15500000.0,3276.96,2022
1,Santa Maria 1643 Brickell Ave PH 4902 Miami,5/10/2021,5/5/2022,309,"$12,000,000.00",10000,"$1,200.00",Audrey Ross,"Compass Florida, LLC",Liz Hogan,...,Santa Maria,1643 Brickell Ave PH 4902 Miami,"(1643 Brickell Ave # 4902, Miami, FL 33129, US...","(25.755254, -80.194744, 0.0)",25.755254,-80.194744,0.0,12000000.0,1200.00,2022
2,Oceana Bal Harbour Condo 10201 Collins Ave 210...,9/22/2021,5/5/2022,201,"$8,900,000.00",3264,"$2,726.72",Jonathan Mann,Coldwell Banker Realty,Omer Horev,...,Oceana Bal Harbour Condo,10201 Collins Ave 2103 Bal Harbour,"(10201 Collins Ave Unit 2103, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0,8900000.0,2726.72,2022
3,Oceana Bal Harbour Condo 10203 Collins Ave 160...,5/1/2022,5/2/2022,1,"$8,900,000.00",3450,"$2,579.71",Comp Sale,Comp Sale,Lourdes Alatriste,...,Oceana Bal Harbour Condo,10203 Collins Ave 1602 Bal Harbour,"(10203 Collins Ave Unit 1602, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0,8900000.0,2579.71,2022
4,Continuum on South Beach 50 S Pointe Dr 3104 M...,4/6/2022,5/2/2022,2,"$8,800,000.00",2173,"$4,049.70",Jorge Martinez,One Sotheby's Intl Realty,Devin Kay,...,Continuum on South Beach,50 S Pointe Dr 3104 Miami Beach,"(50 South Pointe Dr APT 3104, Miami Beach, FL ...","(25.7682956, -80.1316247, 0.0)",25.768296,-80.131625,0.0,8800000.0,4049.70,2022
5,Oceana Key Biscayne 360 Ocean Dr 501S Key Bisc...,8/11/2021,5/5/2022,253,"$8,800,000.00",3791,"$2,321.29",Brigitte Nachtigall,Great Properties Intl,David Deleon,...,Oceana Key Biscayne,360 Ocean Dr 501S Key Biscayne,"(360 Ocean Dr #501s, Key Biscayne, FL 33149, U...","(25.6942877, -80.1578218, 0.0)",25.694288,-80.157822,0.0,8800000.0,2321.29,2022
6,One Thousand Museum 1000 E Biscayne Blvd 3201 ...,4/26/2022,5/4/2022,1,"$7,300,000.00",4600,"$1,586.96",Patrice Hallot,London Foster Realty,Anna Sherrill,...,One Thousand Museum,1000 E Biscayne Blvd 3201 Miami,"(1000 Biscayne Blvd #3201, Miami, FL 33132, US...","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0,7300000.0,1586.96,2022
7,Cloisters on the Bay Condo 3471 Main Hwy 727 M...,2/28/2022,5/2/2022,7,"$7,000,000.00",5739,"$1,219.72",Ilana Levitt,"Compass Florida, LLC",Ilana Levitt,...,Cloisters on the Bay Condo,3471 Main Hwy 727 Miami,"(3471 Main Hwy APT 727, Miami, FL 33133, USA, ...","(25.7260243, -80.2424968, 0.0)",25.726024,-80.242497,0.0,7000000.0,1219.72,2022
8,Bayview at Fisher Island 5132 Fisher Island Dr...,11/16/2021,5/2/2022,121,"$6,500,000.00",3580,"$1,815.64",Riley Smith,BHHS EWM Realty,Riley Smith,...,Bayview at Fisher Island,5132 Fisher Island Dr 5132 Miami Beach,"(5132 Fisher Island Dr #5132, Miami Beach, FL ...","(25.7620721, -80.1477688, 0.0)",25.762072,-80.147769,0.0,6500000.0,1815.64,2022
9,P0orsche Design Tower 18555 Collins Ave 2501 S...,1/11/2022,5/2/2022,51,"$6,050,000.00",5913,"$1,340.57",Enrique Tettamanti,Dezer Platinum Realty,Melissa Barragan,...,P,0orsche Design Tower 18555 Collins Ave 2501 Su...,"(18555 Collins Ave, Sunny Isles Beach, FL 3316...","(25.9482145, -80.1197839, 0.0)",25.948214,-80.119784,0.0,6050000.0,1340.57,2022


In [189]:
df['year'] = '2021'

In [190]:
df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,...,building_name,address_only,loc,point,lat,lon,altitude,Sale Price int,price_per_sqft_int,year
0,Faena House 3315 Collins Ave 6a Miami Beach,1/31/2022,5/4/2022,63,"$15,500,000.00",6246,"$3,276.96",Toni Schrager,Brown Harris Stevens,Bryan Sereny PA,...,Faena House,3315 Collins Ave 6a Miami Beach,"(FAENA HOUSE, 3315 Collins Ave #6a, Miami Beac...","(25.8078488, -80.1232302, 0.0)",25.807849,-80.123230,0.0,15500000.0,3276.96,2021
1,Santa Maria 1643 Brickell Ave PH 4902 Miami,5/10/2021,5/5/2022,309,"$12,000,000.00",10000,"$1,200.00",Audrey Ross,"Compass Florida, LLC",Liz Hogan,...,Santa Maria,1643 Brickell Ave PH 4902 Miami,"(1643 Brickell Ave # 4902, Miami, FL 33129, US...","(25.755254, -80.194744, 0.0)",25.755254,-80.194744,0.0,12000000.0,1200.00,2021
2,Oceana Bal Harbour Condo 10201 Collins Ave 210...,9/22/2021,5/5/2022,201,"$8,900,000.00",3264,"$2,726.72",Jonathan Mann,Coldwell Banker Realty,Omer Horev,...,Oceana Bal Harbour Condo,10201 Collins Ave 2103 Bal Harbour,"(10201 Collins Ave Unit 2103, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0,8900000.0,2726.72,2021
3,Oceana Bal Harbour Condo 10203 Collins Ave 160...,5/1/2022,5/2/2022,1,"$8,900,000.00",3450,"$2,579.71",Comp Sale,Comp Sale,Lourdes Alatriste,...,Oceana Bal Harbour Condo,10203 Collins Ave 1602 Bal Harbour,"(10203 Collins Ave Unit 1602, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0,8900000.0,2579.71,2021
4,Continuum on South Beach 50 S Pointe Dr 3104 M...,4/6/2022,5/2/2022,2,"$8,800,000.00",2173,"$4,049.70",Jorge Martinez,One Sotheby's Intl Realty,Devin Kay,...,Continuum on South Beach,50 S Pointe Dr 3104 Miami Beach,"(50 South Pointe Dr APT 3104, Miami Beach, FL ...","(25.7682956, -80.1316247, 0.0)",25.768296,-80.131625,0.0,8800000.0,4049.70,2021
5,Oceana Key Biscayne 360 Ocean Dr 501S Key Bisc...,8/11/2021,5/5/2022,253,"$8,800,000.00",3791,"$2,321.29",Brigitte Nachtigall,Great Properties Intl,David Deleon,...,Oceana Key Biscayne,360 Ocean Dr 501S Key Biscayne,"(360 Ocean Dr #501s, Key Biscayne, FL 33149, U...","(25.6942877, -80.1578218, 0.0)",25.694288,-80.157822,0.0,8800000.0,2321.29,2021
6,One Thousand Museum 1000 E Biscayne Blvd 3201 ...,4/26/2022,5/4/2022,1,"$7,300,000.00",4600,"$1,586.96",Patrice Hallot,London Foster Realty,Anna Sherrill,...,One Thousand Museum,1000 E Biscayne Blvd 3201 Miami,"(1000 Biscayne Blvd #3201, Miami, FL 33132, US...","(25.784171, -80.1900783, 0.0)",25.784171,-80.190078,0.0,7300000.0,1586.96,2021
7,Cloisters on the Bay Condo 3471 Main Hwy 727 M...,2/28/2022,5/2/2022,7,"$7,000,000.00",5739,"$1,219.72",Ilana Levitt,"Compass Florida, LLC",Ilana Levitt,...,Cloisters on the Bay Condo,3471 Main Hwy 727 Miami,"(3471 Main Hwy APT 727, Miami, FL 33133, USA, ...","(25.7260243, -80.2424968, 0.0)",25.726024,-80.242497,0.0,7000000.0,1219.72,2021
8,Bayview at Fisher Island 5132 Fisher Island Dr...,11/16/2021,5/2/2022,121,"$6,500,000.00",3580,"$1,815.64",Riley Smith,BHHS EWM Realty,Riley Smith,...,Bayview at Fisher Island,5132 Fisher Island Dr 5132 Miami Beach,"(5132 Fisher Island Dr #5132, Miami Beach, FL ...","(25.7620721, -80.1477688, 0.0)",25.762072,-80.147769,0.0,6500000.0,1815.64,2021
9,P0orsche Design Tower 18555 Collins Ave 2501 S...,1/11/2022,5/2/2022,51,"$6,050,000.00",5913,"$1,340.57",Enrique Tettamanti,Dezer Platinum Realty,Melissa Barragan,...,P,0orsche Design Tower 18555 Collins Ave 2501 Su...,"(18555 Collins Ave, Sunny Isles Beach, FL 3316...","(25.9482145, -80.1197839, 0.0)",25.948214,-80.119784,0.0,6050000.0,1340.57,2021


In [191]:
df.to_csv('May2022CondoSales.csv', sep='|')